# HW 2 (due December 5 23:59 MSK)

## IMPORTANT

Although I've written the function to generate big random prime numbers and it works just fine, all prime numbers used in the encryption are relatively small due to the computation cost. I've yet to learn optimal algorithms =(

### Problem 1 (1 points)

Diffie–Hellman key exchange protocol is one of the earliest practical examples of public key exchange implemented within the field of cryptography. Published in 1976 by Diffie and Hellman, this is the earliest publicly known work that proposed the idea of a private key and a corresponding public key. 

1. Implement function to generate common secret key within multiplicative group of given Finite field with known generator 

In [1]:
import math
import numpy as np
import random as rnd

First, let's implement the prime number generator:

In [2]:
def first_primes(num):
  nums = np.arange(2, num)
  for n in range(2, num):
    check_array = nums[nums >= n**2]
    check_array = check_array[check_array % n == 0]
    nums = np.setdiff1d(nums, check_array)

  return nums

first_primes(1000)

def generate_candidate(n):
  return rnd.randrange(2**(n-1), 2**n - 1)

def llpt(candidate, primes):
  for d in primes:
    if candidate % d == 0:
      return False
    return True

def generate_prime(n):
    found = False
    primes = first_primes(1000)
    while not(found):
        c = generate_candidate(n)
        found = llpt(c, primes)
    return c

In [3]:
prime = generate_prime(100)

print('Found prime candidate: {}'.format(prime))

Found prime candidate: 1129855095612002158682989998831


In [4]:
def generate_sk(g, p):
    a = generate_prime(4)
    b = generate_prime(4)

    A = (int(g) ** int(a)) % int(p)
    B = (int(g) ** int(b)) % int(p)

    K1 = (int(B) ** int(a)) % int(p)
    K2 = (int(A) ** int(b)) % int(p)

    K = K1 if (K1 == K2) else print('something went wrong!')
    return K, K1, K2, A, B, a, b

Now, we can generate the key:

In [5]:
K, *tmp = generate_sk(11, 17)
K

11

2. Test your solution in GF(17) with generator g=11. Bobs' open key B=11, Alice private key is a=7

In [6]:
def generate_sk_2(a, B, g, p):
    K = (int(B) ** int(a)) % int(p)
    return K

In [7]:
print('Secret key: {}'.format(generate_sk_2(7, 11, 11, 17)))
print('Check: {}'.format(11 ** 7 % 17))

Secret key: 3
Check: 3


### Problem 2 (3 points)

RSA is a public-key cryptosystem that is widely used for secure data transmission. It is also one of the oldest. In this task we will ask you to implement your own RSA encryption scheme on Python

1. Implement function for generating keys. The function must generate big random prime number (problem of generating big prime numbers was discussed within the lectures). (1 point)

In [8]:
## done in p1

2. Implement functions that realize the encryption and decryption in RSA protocol. (1 points)

In [8]:
def generate_keys():
  P1 = generate_prime(4)
  P2 = generate_prime(4)
  while (P2 == P1):
    P2 = generate_prime(4)

  N = P1 * P2
  Phi = (P1 - 1)*(P2 - 1)
  
  es = first_primes(Phi)
  e = 0
  for ee in es:
    if Phi % ee != 0:
      e = ee
      break

  d = 1
  while ((d * e) % Phi != 1):
    d += 1

  if d != int(d):
    print('You fucked up!')
    return None, None

  return (N, e), (d, P1, P2, Phi)

def encrypt(public, message):
  N, e = public
  #print('Encrypting')
  #print('message: {}, e: {}, N: {}'.format(message, e, N))
  m = int(math.pow(message, e) % N)
  return m

def decrypt(private, public, m): 
  d = private[0]
  N = public[0]
  #print('Decrypting')
  #print('m: {}, d: {}, N: {}'.format(m, d, N))
  message = m ** d % N
  return message

In [15]:
public, private = generate_keys()
public, private

((117, 5), (77, 13, 9, 96))

Test check:

In [16]:
message = 6
assert message < public[0]
print('Message: ', message)
encrypted_message = encrypt(public, message)
print(encrypted_message)
decrypted_message = decrypt(private, public, encrypted_message)
print('Decrypted message: ', decrypted_message)

Message:  6
54
Decrypted message:  45


3. Calculate Hash of your name by SHA-1 and test RSA encryption/decryption functions on it (1 points)

In [14]:
from hashlib import sha1
message = sha1()
message.update(b'PrincessLuna')
message = int(message.hexdigest(), 16)
print('Message: ', message)
message = np.array([int(x) for x in str(message)])
print('Decimal encoded name: {}'.format(message))
encrypted = [encrypt(public, m) for m in message]
print(encrypted)
decrypted = [decrypt(private, public, e) for e in encrypted]
print('Decrypted message: {}'.format(decrypted))

Message:  373441502431982517244888483880274664732014556166
Decimal encoded name: [3 7 3 4 4 1 5 0 2 4 3 1 9 8 2 5 1 7 2 4 4 8 8 8 4 8 3 8 8 0 2 7 4 6 6 4 7
 3 2 0 1 4 5 5 6 1 6 6]
Encrypting
message: 3, e: 7, N: 143
Encrypting
message: 7, e: 7, N: 143
Encrypting
message: 3, e: 7, N: 143
Encrypting
message: 4, e: 7, N: 143
Encrypting
message: 4, e: 7, N: 143
Encrypting
message: 1, e: 7, N: 143
Encrypting
message: 5, e: 7, N: 143
Encrypting
message: 0, e: 7, N: 143
Encrypting
message: 2, e: 7, N: 143
Encrypting
message: 4, e: 7, N: 143
Encrypting
message: 3, e: 7, N: 143
Encrypting
message: 1, e: 7, N: 143
Encrypting
message: 9, e: 7, N: 143
Encrypting
message: 8, e: 7, N: 143
Encrypting
message: 2, e: 7, N: 143
Encrypting
message: 5, e: 7, N: 143
Encrypting
message: 1, e: 7, N: 143
Encrypting
message: 7, e: 7, N: 143
Encrypting
message: 2, e: 7, N: 143
Encrypting
message: 4, e: 7, N: 143
Encrypting
message: 4, e: 7, N: 143
Encrypting
message: 8, e: 7, N: 143
Encrypting
message: 8, e: 7,

### Problem 3 (3 points)

El Gamal protocol is widely used in cryptography. In this task we will ask you to implement your own El-Gamal encryption scheme on Python

1. Implement function for generating keys. The function must generate big random prime number (problem of generating big prime numbers was discussed within the lectures). (1 point)

In [1]:
# solved in p1

2. Implement functions that realize the encryption and decryption in El Gamal protocol. (1 points)

In [42]:
def primRoots(p):
    roots = []
    required_set = set(num for num in range (1, p) if math.gcd(num, p) == 1)

    for g in range(1, p):
        actual_set = set(pow(g, powers) % p for powers in range (1, p))
        if required_set == actual_set:
            roots.append(g)           
    return roots

In [46]:
def el_gamal_keys():
    p = generate_prime(5)
    g = rnd.choice(primRoots(p))
    x = rnd.randrange(2, p - 2)

    y = g ** x % p

    return (y, g, p), x

In [48]:
public, private = el_gamal_keys()
public, private

((15, 14, 17), 6)

In [31]:
def el_encrypt(public, M):
    y, g, p = public
    k = 2
    while math.gcd(k, p-1) != 1:
        k += 1
    
    if (k >= p - 1):
        print('something went wrong')

    a = g ** k % p
    b = (y ** k * M) % p

    return (a, b)

def el_decrypt(private, public, cypher):
    x = private
    a, b = cypher
    y, g, p = public

    #M = b*(a ** (p - 1 - x)) % p
    M = b*(a ** (p - 1 - x)) % p

    return M

In [60]:
M = 6
assert M < public[2]
print('Message: {}'.format(M))
cypher = el_encrypt(public, M)
print(cypher)

msg = el_decrypt(private, public, cypher)

print('Decrypted message: {}'.format(msg))

Message: 6
(7, 3)
Decrypted message: 6


3. Calculate Hash of your name by SHA-1 and test El Gamal encryption/decryption functions on it (1 points)

In [62]:
from hashlib import sha1
message = sha1()
message.update(b'PrincessLuna')
message = int(message.hexdigest(), 16)
print('Message: ', message)
message = np.array([np.int64(x) for x in str(message)])
print('Decimal encoded name: {}'.format(message))
encrypted = [el_encrypt(public, m) for m in message]
print(encrypted)
decrypted = [el_decrypt(private, public, e) for e in encrypted]
string = ''
for d in decrypted:
    string += str(d)

print('Decrypted message: {}'.format(string))

Message:  373441502431982517244888483880274664732014556166
Decimal encoded name: [3 7 3 4 4 1 5 0 2 4 3 1 9 8 2 5 1 7 2 4 4 8 8 8 4 8 3 8 8 0 2 7 4 6 6 4 7
 3 2 0 1 4 5 5 6 1 6 6]
[(7, 10), (7, 12), (7, 10), (7, 2), (7, 2), (7, 9), (7, 11), (7, 0), (7, 1), (7, 2), (7, 10), (7, 9), (7, 13), (7, 4), (7, 1), (7, 11), (7, 9), (7, 12), (7, 1), (7, 2), (7, 2), (7, 4), (7, 4), (7, 4), (7, 2), (7, 4), (7, 10), (7, 4), (7, 4), (7, 0), (7, 1), (7, 12), (7, 2), (7, 3), (7, 3), (7, 2), (7, 12), (7, 10), (7, 1), (7, 0), (7, 9), (7, 2), (7, 11), (7, 11), (7, 3), (7, 9), (7, 3), (7, 3)]
Decrypted message: 373441502431982517244888483880274664732014556166


It works!

### Problem 4 (3 points)

Elliptic curves due to their efficient hardware realization widely used in modern secure communication channels. The main thing that lies inside their cryptographic hardness is that we can break them only by greed search over all group points. In this task, we will ask you to write python function that returns all group elements of a certain elliptic curve over a finite field 

1. Write a python function that list all points of elliptic curve $y^2=x^3-8x-5$ over $F_{11}$ (2 points)

2. Does the point (9, 5) generate all elliptic curve points or only its' subgroup? Provide a python fuction that solve this task. (1 point)